### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [21]:
KEYWORDS = ['python', 'парсинг']

In [22]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [23]:
posts = soup.find_all('article', class_='post post_preview')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/alexzfort/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/d2a/e8a/5ac/d2ae8a5ac9876f838ebdd45f3ea3bc3b.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">alexzfort</span>
 </a>
 <span class="post__time">сегодня в 00:35</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/537782/">Дайджест свежих материалов из мира фронтенда за последнюю неделю №450 (11 — 17 января 2021)</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/webdev/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Разработка веб-сайтов'); }" rel="nofo

In [24]:
posts_filtred = []
for post in posts:
    for key in KEYWORDS:
        if key in post.text.lower():
            posts_filtred.append(post)
            break

In [25]:
habr_request = pd.DataFrame()

for post in posts_filtred:
    date = post.find('span', class_='post__time').text 
    title = post.find('a', class_='post__title_link').text
    link = post.find('a', class_ = 'post__title_link').get('href')   

    
    row = {'date': date, 'title': title, 'link' : link}
    habr_request = pd.concat([habr_request, pd.DataFrame([row])])
#                 habr_news = pd.concat([habr_news, pd.DataFrame([row])], ignore_index=True)
habr_request.head()

,date,title,link
0,вчера в 22:37,Многопоточное скачивание файлов с ftp python-с...,https://habr.com/ru/post/537774/
0,вчера в 21:54,Нейросеть для раскрутки собачьего аккаунта в И...,https://habr.com/ru/post/537714/
0,вчера в 18:27,Прокачиваем скрипты симуляции HDL с помощью Py...,https://habr.com/ru/post/537704/


### Задание 2.

Обязательная часть  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [26]:
EMAIL =  ['xxx@x.ru', 'yyy@y.com']

In [27]:
url = 'https://identityprotection.avast.com/v1/web/query/breaches-stats'
headers = {
# 'Accept': 'application/json, text/plain, */*',
# 'Accept-Encoding': 'gzip, deflate, br',
# 'Accept-Language': 'ru,en;q=0.9',
# 'Connection': 'keep-alive',
# 'Content-Length': '31',
# 'Host': 'identityprotection.avast.com',
# 'Origin': 'https://www.avast.com',
# 'Referer': 'https://www.avast.com/hackcheck/',
# 'Sec-Fetch-Dest': 'empty',
# 'Sec-Fetch-Mode': 'cors',
# 'Sec-Fetch-Site': 'same-site',
# 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.60 YaBrowser/20.12.0.966 Yowser/2.5 Safari/537.36',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [28]:
params = {
    'emailAddresses': EMAIL
}

In [29]:
res = requests.post(url,json = params,headers=headers)

In [30]:
res.json()

In [31]:

for email in EMAIL:
    res = requests.post(url, data = email, headers = headers)
    for leak in res.json()['biggest']:
        df = pd.concat([df, pd.DataFrame([{ 'email'         : email,
                                            'Дата '   : leak['publishDate'],
                                            'Источник' : leak['site'],
                                            'Описание'   : leak['description']
                                                      }])])
df.head()

TypeError: 'NoneType' object is not subscriptable

In [32]:
def find_breaches(mail):

    time.sleep(0.3)
    json_mail = {'emailAddresses': [mail]}
    req = requests.post(url, json = json_mail, headers = headers)    
    req_df = pd.DataFrame(req.json())

    breaches_count = len(req_df.index)//2
    mail_breaches_df = pd.DataFrame(columns=['mail', 'date', 'site', 'description'])  
    
    if breaches_count == 0:
        row = {'mail': mail, 'date' : None, 'site' : None, 'description' : None}
        mail_breaches_df = mail_breaches_df.append(pd.DataFrame([row]))  
    else: 
        for i in range(breaches_count):
            br = req_df.iloc[i]['breaches']
            publishDate = br['publishDate']
            site = br['site']
            description = br['description']
            row = {'mail': mail, 'date' : publishDate, 'site' : site, 'description' : description}
            mail_breaches_df = mail_breaches_df.append(pd.DataFrame([row]))
    
    return mail_breaches_df

breaches_df = pd.DataFrame(columns=['mail', 'date', 'site', 'description'])

for mail in EMAIL:
    mail_breaches_df = find_breaches(mail)
    breaches_df = pd.concat([breaches_df, mail_breaches_df], ignore_index=True)

breaches_df.head()

,mail,date,site,description
0,xxx@x.ru,None,None,None
1,yyy@y.com,None,None,None
